# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [92]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [93]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Knee', 'Left', 'Sit', 'StandInit', 'HeadBack', 'Right', 'Frog', 'Back', 'Belly', 'Crouch', 'Stand']


In [94]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [95]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
list_length = len(classes)
for i in range(list_length):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
print ('total number of data', len(all_data))
print ('total number of target', len(all_target))

total number of data 222
total number of target 222


In [96]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
print ('training data length:', len(training_data))

training data length: 155


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [97]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [98]:
# YOUR CODE HERE
sample_data = np.asarray(all_data)[training_data]
reference_data = np.asarray(all_target)[training_data]
clf.fit(sample_data, reference_data)

SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### predicting

In [99]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])

In [100]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [103]:
expected = reference_data
predicted = clf.predict(sample_data)


evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00        34
           4       1.00      1.00      1.00         5
           5       1.00      0.89      0.94         9
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        16
           8       1.00      1.00      1.00        12
           9       0.95      1.00      0.98        20
          10       1.00      1.00      1.00        11

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  0  0]
 [ 0  0  0 34  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [104]:
test_data = permutation[n_training_data:]
expected = np.asarray(all_target)[test_data]
predicted = clf.predict(np.asarray(all_data)[test_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        18
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         7
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00        10

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 3  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [107]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [108]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])